# Week 01 - Lesson 03 - Foundations of AI Agent Development

This notebook provides hands-on demonstrations of AI agent concepts introduced in `Week 01`. 

### Learning Objectives
By the end of this notebook, you will:
1. Understand the fundamental components of AI agents
2. Create your first functional AI agent
3. Implement memory and structured outputs
4. Understand practical implications

## Requirements

- **Python** installed (https://www.python.org/downloads/)
- **Jupyter Notebook** (already installed in this environment)
- **OpenAI API Key** (https://platform.openai.com/api-keys)
- The following Python libraries:
  - `openai` (for LLM access)
  - `langchain` (for agent and tool orchestration)
  - `langgraph` (for agent workflow management)

## 1. Environment Setup and Dependencies

First, let's ensure we have all necessary dependencies installed and configured.

In [ ]:
# Install required dependencies
%pip install -U --quiet langgraph langchain pydantic langchain-openai

In [ ]:
# Your OpenAI API Key here
OPENAI_API_KEY = "sk-your-openai-api-key-here"

## Lesson: From Pre-built to Custom

### Why We Start with Pre-built Libraries

In this lesson, we'll begin our AI agent journey using LangChain's **`create_react_agent`** function from the [Langgraph agents library](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.AgentState). This approach serves several important purposes:


<div style="width:70%;overflow:hidden;">
  <img src="https://cdn-blog.scalablepath.com/uploads/2025/06/header-image-langgraph.png" 
       style="width:100%;display:block;object-fit:cover;object-position:top;aspect-ratio:1200/600;max-height:55%;">
</div>

#### **Learning Benefits**
- **Faster Understanding**: Pre-built agents handle complex orchestration automatically
- **Focus on Concepts**: We can concentrate on understanding agent behavior rather than implementation details
- **Immediate Results**: See working agents in action without building infrastructure from scratch

#### **What `create_react_agent` Provides**
The `create_react_agent` function implements the **ReAct (Reasoning + Acting)** paradigm, which:
- Combines logical reasoning with tool execution
- Manages the thought process automatically
- Handles tool selection and parameter passing
- Maintains conversation context and memory

---

### **Progressive Learning Path**

#### **Phase 1: Understanding (Current Lesson)**
- Use `create_react_agent` to grasp fundamental concepts
- Learn how agents think, reason, and act
- Understand tool integration and memory management

#### **Phase 2: Customization (Future Lessons)**
- Build agents from scratch without pre-built agent libraries
- Design custom workflows and decision trees (next lesson)
- Implement specialized reasoning patterns
- Create orchestration systems

#### **Phase 3: Advanced Implementation**
- Develop multi-agent systems
- Create custom state management
- Implement advanced memory strategies
- Build production-ready agent architectures

---

### **Key Takeaway**

Starting with pre-built libraries like `create_react_agent` is not a limitation—it's a strategic learning approach. By understanding how these agents work internally, you'll be better equipped to build custom solutions that perfectly fit your enterprise requirements.

In [3]:
# Import necessary libraries
from pydantic import BaseModel

# LangGraph and LangChain imports
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

print("✅ All dependencies imported successfully!")

✅ All dependencies imported successfully!


## 2. Understanding AI Agent Architecture

Before diving into code, let's understand the fundamental architecture of AI agents:

### Core Components

1. **Language Model (LLM)**: The "brain" that processes natural language and makes decisions
2. **Tools**: External functions the agent can call to perform actions
3. **Memory**: Storage for conversation history and context
4. **Prompt**: Instructions that define the agent's behavior and capabilities
5. **Workflow Engine**: Manages the flow between thinking, tool usage, and responses

## 3. Creating Your First AI Agent

Let's start with a simple example that demonstrates the basic agent creation process.

In [4]:
# Now we can initialize the LLM
llm = init_chat_model(
    "openai:gpt-4o-mini",
    api_key=OPENAI_API_KEY,
)

## Before we begin, let's try to understand a very important concept, superficially:

### What are Tools?

**Tools** are the fundamental components that transform a Language Model from a conversational interface into a true **AI Agent**. Think of tools as the "hands" of your agent - they allow the agent to interact with external systems, perform actions, and access real-world data.

### Key Characteristics of Tools

- Tools are **functions** that the agent can call
- The agent decides **when** and **how** to use these tools based on user requests
- Tools connect your agent to databases, APIs, file systems, and other services
- They enable the agent to perform actions beyond text generation
- The agent **automatically determines** which tools to use

### **Important Note for This Lesson**

**We will not dive deep into tool implementation details or function calling mechanisms in this lesson.** Our focus is on understanding the **conceptual role** of tools in agent architecture.

**In Week 02**, we will explore topics such as detailed tool creation and configuration, advanced function calling patterns, custom tool development for enterprise scenarios, and the orchestration and management of workflows involving multiple tools.

For now, simply understand that tools are functions which agents can call. They enable interaction with external systems, transforming LLMs into actionable AI agents. This capability is essential for building enterprise-grade AI solutions.

*Let's see tools in action with our first practical examples...*


In [5]:
# Define a simple tool function
# In real-world applications, these tool functions would connect to external APIs or databases.
def get_weather(city: str) -> str:
    """Get weather information for a given city.
    
    This is a mock implementation. In production, this would call a weather API.
    """
    return f"The weather in {city} is currently sunny with a temperature of 22°C."

def get_stock_price(symbol: str) -> str:
    """Get current stock price for a given symbol.
    
    This is a mock implementation. In production, this would call a financial API.
    """
    return f"The current stock price of {symbol} is $150.00."

print("✅ Tool functions defined successfully!")

✅ Tool functions defined successfully!


In [6]:
# Create your first agent
# This demonstrates the basic pattern for agent creation
agent = create_react_agent(
    model=llm,  # The LLM to use
    tools=[get_weather, get_stock_price],         # Available tools
    prompt="You are a helpful business assistant that can provide weather and stock information."
)

print("✅ Agent created successfully!")
print(f"Agent has {len([get_weather, get_stock_price])} tools available")

✅ Agent created successfully!
Agent has 2 tools available


In [7]:
# Test the agent with a simple query
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]}
)

print("🤖 Agent Response:")
print(response["messages"][-1].content)

🤖 Agent Response:
The weather in San Francisco is currently sunny with a temperature of 22°C.


## 4. Agent with more Sophisticated Configuration

Now let's explore more advanced configurations:

In [8]:
# Configure an LLM with parameters
# This demonstrates how to fine-tune the model behavior
model = init_chat_model(
    "openai:gpt-4o-mini",
    api_key=OPENAI_API_KEY,
    temperature=0.1,  # Low temperature for consistent, predictable responses
    max_tokens=1000   # Limit response length for cost control
)

# Create a more sophisticated agent
# Compare to the previous agent, a more well defined prompt leads to better results and more consistent responses that can call tools more accurately
sophisticated_agent = create_react_agent(
    model=model,
    tools=[get_weather, get_stock_price],
    prompt="""
    You are an enterprise business assistant with the following responsibilities:
    
    1. Provide accurate and concise information
    2. Always cite your sources when using tools
    3. Maintain professional communication
    4. Prioritize data accuracy over speculation
    
    Available tools:
    - get_weather: Provides weather information for cities
    - get_stock_price: Provides current stock prices
    """
)

print("✅ Sophisticated agent configured successfully!")

✅ Sophisticated agent configured successfully!


In [9]:
# Test the sophisticated agent
response = sophisticated_agent.invoke(
    {"messages": [{"role": "user", "content": "Can you check the weather in New York and the stock price of AAPL?"}]}
)

print("🏢 Sophisticated Agent Response:")
print(response["messages"][-1].content)

🏢 Sophisticated Agent Response:
The current weather in New York is sunny with a temperature of 22°C. Additionally, the current stock price of Apple Inc. (AAPL) is $150.00.


## 5. Implementing Memory and State Management

Memory is crucial for applications where context must be maintained across multiple interactions.

In [10]:
# Set up memory management for the agent
# This enables multi-turn conversations and context preservation
checkpointer = InMemorySaver()

agent_with_memory = create_react_agent(
    model=model,
    tools=[get_weather, get_stock_price],
    checkpointer=checkpointer,  # Enable memory
    prompt="You are a helpful business assistant. Remember previous conversations and build on them."
)

print("✅ Agent with memory configured successfully!")

✅ Agent with memory configured successfully!


In [11]:
# Demonstrate multi-turn conversation with memory
config = {"configurable": {"thread_id": "session_001"}}

# First interaction
print("🔄 First Interaction:")
response1 = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in London?"}]},
    config
)
print(response1["messages"][-1].content)
print("\n" + "="*50 + "\n")

# Second interaction (agent remembers the context)
print("🔄 Second Interaction (with memory):")
response2 = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "How does that compare to the weather in Paris?"}]},
    config
)
print(response2["messages"][-1].content)

🔄 First Interaction:
The weather in London is currently sunny with a temperature of 22°C.


🔄 Second Interaction (with memory):
The weather in Paris is also sunny with a temperature of 22°C, which is the same as in London.


## 6. Structured Outputs

Structured outputs are essential for systems that need to integrate with existing databases, APIs, and workflows.

In [12]:
# Define structured output schemas using Pydantic
# This ensures consistent, predictable data formats
class WeatherReport(BaseModel):
    city: str
    temperature: str
    conditions: str
    timestamp: str

class StockReport(BaseModel):
    symbol: str
    price: str
    currency: str
    timestamp: str

class BusinessReport(BaseModel):
    weather: WeatherReport
    stock: StockReport
    summary: str

print("✅ Structured output schemas defined!")

✅ Structured output schemas defined!


In [13]:
# Create an agent with structured outputs
structured_agent = create_react_agent(
    model=model,
    tools=[get_weather, get_stock_price],
    response_format=BusinessReport,  # Enforce structured output
    prompt="""
    You are a business intelligence agent. When asked about weather and stocks,
    provide structured reports that can be easily integrated into business systems.
    
    Always format your responses according to the BusinessReport schema.
    """
)

print("✅ Structured output agent created!")

✅ Structured output agent created!


In [14]:
# Test structured output
response = structured_agent.invoke(
    {"messages": [{"role": "user", "content": "Provide a business report for Tokyo weather and TSLA stock price"}]}
)

print("📊 Structured Business Report:")
structured_data = response["structured_response"]
print(f"Weather: {structured_data.weather.city} - {structured_data.weather.temperature}, {structured_data.weather.conditions}")
print(f"Stock: {structured_data.stock.symbol} - {structured_data.stock.price} {structured_data.stock.currency}")
print(f"Summary: {structured_data.summary}")

print("\n🔍 Raw Structured Data (for system integration):")
print(structured_data.model_dump())

📊 Structured Business Report:
Weather: Tokyo - 22°C, Sunny
Stock: TSLA - $150.00 USD
Summary: Current weather in Tokyo is sunny with a temperature of 22°C. TSLA stock is currently priced at $150.00.

🔍 Raw Structured Data (for system integration):
{'weather': {'city': 'Tokyo', 'temperature': '22°C', 'conditions': 'Sunny', 'timestamp': '2023-10-10T12:00:00Z'}, 'stock': {'symbol': 'TSLA', 'price': '$150.00', 'currency': 'USD', 'timestamp': '2023-10-10T12:00:00Z'}, 'summary': 'Current weather in Tokyo is sunny with a temperature of 22°C. TSLA stock is currently priced at $150.00.'}


## 7. Integration

In [15]:
# Example: tool integration
# In real scenarios, these would be actual API calls (this is a mock implementation)
def get_customer_data(customer_id: str) -> str:
    """Retrieve customer information from CRM system."""
    return f"Customer {customer_id}: Premium tier, last purchase: 2024-01-15, total spend: $2,500"

def get_sales_forecast(region: str) -> str:
    """Get sales forecast from business intelligence system."""
    return f"Sales forecast for {region}: Q1 2024 - $1.2M, Q2 2024 - $1.5M"

def update_crm_record(customer_id: str, action: str) -> str:
    """Update customer record in CRM system."""
    return f"Updated CRM record for customer {customer_id} with action: {action}"

print("✅ Tools defined!")

✅ Tools defined!


In [16]:
# Create an business agent
business_agent = create_react_agent(
    model=model,
    tools=[get_customer_data, get_sales_forecast, update_crm_record],
    checkpointer=InMemorySaver(),
    prompt="""
    You are an enterprise business operations agent with access to:
    - Customer relationship management (CRM) system
    - Sales forecasting and business intelligence
    - Customer data management capabilities
    
    Your role is to assist with business operations, customer service, and data analysis.
    Always maintain data privacy and follow enterprise security protocols.
    """
)

print("✅ Business agent created!")

✅ Business agent created!


In [17]:
# Demonstrate workflow
config = {"configurable": {"thread_id": "business_001"}}

print("🏢 Workflow Example:")
print("=" * 60)

# Multi-step process
steps = [
    "Can you get information about customer CUST-001?",
    "What's the sales forecast for the North region?",
    "Based on this information, update the CRM record for CUST-001 with a follow-up action"
]

# This is a mock implementation of a multi-step process
# In real applications, each step would be a separate agent invocation
for i, step in enumerate(steps, 1):
    print(f"\n📋 Step {i}: {step}")
    response = business_agent.invoke(
        {"messages": [{"role": "user", "content": step}]},
        config
    )
    print(f"🤖 Response: {response['messages'][-1].content}")
    print("-" * 40)

🏢 Workflow Example:

📋 Step 1: Can you get information about customer CUST-001?
🤖 Response: Customer CUST-001 is a premium tier customer. Their last purchase was on January 15, 2024, and their total spend is $2,500.
----------------------------------------

📋 Step 2: What's the sales forecast for the North region?
🤖 Response: The sales forecast for the North region is as follows:
- Q1 2024: $1.2 million
- Q2 2024: $1.5 million
----------------------------------------

📋 Step 3: Based on this information, update the CRM record for CUST-001 with a follow-up action
🤖 Response: The CRM record for customer CUST-001 has been updated with the follow-up action: "Follow-up on recent purchase and discuss potential upsell opportunities."
----------------------------------------


## 8. Key Takeaways

### What We've Accomplished

1. **Basic Agent Creation**: Established the fundamental pattern for creating AI agents
2. **Configuration**: Demonstrated how to configure agents for basic business use
3. **Memory Management**: Implemented context preservation across conversations
4. **Structured Outputs**: Created predictable data formats for system integration
5. **Integration**: Showed how agents can interact with business systems

### Enterprise Architecture Considerations

#### Scalability
- Agents can be deployed as microservices
- Memory can be distributed across multiple instances
- Tools can be scaled independently

#### Security
- API keys must be managed securely
- Tool access should be restricted based on user permissions
- Memory storage should comply with data privacy regulations

#### Integration
- Agents can replace or augment traditional business process automation
- Structured outputs enable seamless integration with existing systems
- Memory enables complex, multi-step business workflows

#### Cost Management
- Token usage should be monitored and optimized
- Response length limits help control costs
- Tool usage can be tracked for billing purposes

### Common Use Cases

- **Customer Service**: Automated support with human escalation
- **Sales Operations**: Lead qualification and follow-up automation
- **Data Analysis**: Automated reporting and insights generation
- **Process Automation**: Complex workflow orchestration
- **Knowledge Management**: Intelligent document search and summarization

## 9. Exercise: Build Your First Agent

Now it's your turn to create an agent for a specific business scenario. Choose one of the following exercises:

In [18]:
# Exercise 1: Customer Service Agent
# Create an agent that can handle customer inquiries and update customer records

def get_order_status(order_id: str) -> str:
    """Get the status of a customer order."""
    return f"Order {order_id}: Shipped on 2024-01-20, expected delivery 2024-01-25"

def create_support_ticket(customer_id: str, issue: str) -> str:
    """Create a support ticket for a customer issue."""
    return f"Support ticket created for customer {customer_id}: {issue}"

# TODO: Create your customer service agent here
# Hint: Use create_react_agent with the tools above

print("🎯 Exercise 1: Customer Service Agent")
print("Create an agent that can:")
print("- Check order status")
print("- Create support tickets")
print("- Provide helpful customer service responses")

🎯 Exercise 1: Customer Service Agent
Create an agent that can:
- Check order status
- Create support tickets
- Provide helpful customer service responses


In [19]:
# Exercise 2: Data Analysis Agent
# Create an agent that can analyze business data and generate reports

def get_sales_data(period: str) -> str:
    """Get sales data for a specific period."""
    return f"Sales data for {period}: Revenue $500K, Units sold 1,250, Growth 15%"

def get_customer_satisfaction(region: str) -> str:
    """Get customer satisfaction scores for a region."""
    return f"Customer satisfaction for {region}: 4.2/5.0, Response rate 85%"

# TODO: Create your data analysis agent here
# Hint: Use structured outputs for consistent report formats

print("📊 Exercise 2: Data Analysis Agent")
print("Create an agent that can:")
print("- Retrieve sales data")
print("- Analyze customer satisfaction")
print("- Generate structured business reports")

📊 Exercise 2: Data Analysis Agent
Create an agent that can:
- Retrieve sales data
- Analyze customer satisfaction
- Generate structured business reports


## 10. Conclusion

Congratulations! You've successfully completed the foundational week of AI agent development. You now understand:

### Technical Competencies
- How to create and configure basic AI agents
- How to implement memory and state management
- How to structure outputs
- How to integrate agents with business systems

### Insights
- The architectural implications of AI agents
- How agents can transform business processes
- Common use cases and implementation patterns

---

Happy coding! 🚀